In [102]:
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import re

sectors_physics_and_astronomy={
'Banen in construction':'https://www.glassdoor.com/Job/construction-and-architecture-jobs-jobs-SRCH_KO0,34_IP2.htm?includeNoSalaryJobs=true&seniorityType=entrylevel&pgc=AB4AAYEAHgAAAAAAAAAAAAAAAgU2Q9cAMgEBAQkEtbn7H%2BQCd3WqcQgm0Jy3FwEYgMlg7wU7EyhJG%2FU3U62P%2BgJKhXtDO89CrEmVAAA%3D', 
}                       #https://www.glassdoor.com/Job/jobs.htm?sc.occupationParam=production%2C+asse_IP2.htm?includeNoSalaryJobs=true&seniorityType=entrylevel&pgc=AB4AAYEAHgAAAAAAAAAAAAAAAgVo%2BhAAQwEBAQ4B%2FuAyRqCy9tjSn9T0XBZLeI6jsOcca0fQbqzFYMHfit6cggVisKzDOEjo92fdmD4f%2F4cdrxGsu29OSW%2BvrAYAAA%3D%3D
for j in range(20, 25):
    options = Options()
    options.add_argument("user-data-dir=C:\\Users\\Username\\AppData\\Local\\Google\\Chrome\\User Data\\Person 3")
    driver = webdriver.Chrome(options=options,executable_path='C:/User/fayyo/chromedriver')
    job_desc_Math_teacher={}
            #https://www.glassdoor.com/Job/jobs.htm?sc.occupationParam=production%2C+asse_IP2.htm?includeNoSalaryJobs=true&seniorityType=entrylevel&pgc=AB4AAYEAHgAAAAAAAAAAAAAAAgVo%2BhAAQwEBAQ4B%2FuAyRqCy9tjSn9T0XBZLeI6jsOcca0fQbqzFYMHfit6cggVisKzDOEjo92fdmD4f%2F4cdrxGsu29OSW%2BvrAYAAA%3D%3D
    site = f"https://www.glassdoor.com/Job/netherlands-healthcare-jobs-SRCH_IL.0,11_IN178_KO12,22_IP{j}.htm?includeNoSalaryJobs=true&pgc=AB4AAYEAHgAAAAAAAAAAAAAAAgWvFowARQEBAQcAgJErRjhxgzQ7VyJk6U6G1q1AxEue5W31ev3s9b9vAaMmaPnLKT6jijS1sUktCAYLBvsJUlscJRtbfJFMP5EzcQAA"
    # site = list(sectors_physics_and_astronomy.values())[j]
    driver.get(site)
    time.sleep(20)
    initialScroll = 0
    finalScroll = 1000
    start = time.time()
    while True:
        driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
        initialScroll = finalScroll
        finalScroll += 1000
        time.sleep(3)
        end = time.time()
        if round(end - start) > 25:
            break
    try:
        for i in range(1, 30):                          #'/html/body/div[2]/div/div/div/div/div[2]/section/article/div[1]/ul/li[1]/div/div/a'
            job_desc_butt=driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div[2]/section/article/div[1]/ul/li['+str(i)+']')
            job_desc_butt.click()
            time.sleep(5)
            src = driver.page_source
            soup = BeautifulSoup(src, 'lxml')
            job_descript=soup.find('div', {'class':'jobDescriptionContent desc'})
            job_description=job_descript.get_text()
            job_desc_Math_teacher[i]=job_description
    except Exception as e:
        print(e)
    soup = BeautifulSoup(src, 'lxml')
    src = driver.page_source
    jobs=soup.find_all('li',{'class':re.compile("react-job-listing css")})

    images=[]
    salaries=[]
    companies=[]
    job_titles=[]
    locations=[]

    for pro in jobs[1:30]:
        try:
            image=pro.find('img')['src']
            images.append(image)
        except Exception as e:
            print(f"images_error--> {e}")
            images.append("nan")
        try:
            salary=pro.find('div', {"class":"salary-estimate"})
            # print(salary)
            salaries.append(salary.text.strip())
        except Exception as e:
            print(f"salaries_error--> {e}")
            salaries.append("nan")
        try:
            company=pro.find('div', {"class":"ml-xsm job-search-1bgdn7m"}).text.strip()
            # print(company)
            companies.append(company)
        except Exception as e:
            print(f"companies_error--> {e}")
            companies.append("nan")
        try:
            job=pro.find('div', {"class":"job-title mt-xsm"})
            # print(job)
            job_titles.append(job.text.strip())
        except Exception as e:
            print(f"job_titles_error--> {e}")
            job_titles.append("nan")
        try:
            loc=pro.find('div', {"class":"location"})
            locations.append(loc.text.strip())
        except Exception as e:
            print(f"locations_error--> {e}")
            locations.append("nan")
    print("creating df")
    df_teache=pd.DataFrame({
        "job_titles":job_titles,
        "job_description":list(job_desc_Math_teacher.values()),
        "images":images,
        "locations":locations,
        "salaries":salaries,
        "companies":companies
        }
    )
    df_teache.to_csv(f"healthcare{j}.csv", sep=',', index=False)
    driver.close()

C:\Users\fayyo\AppData\Local\Temp\ipykernel_16200\1734802755.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path='C:/User/fayyo/chromedriver')
Incompatible release of chromedriver (version 111.0.5563.64) detected in PATH: c:\Users\GitHub\Bots_using_selenium\linkedin_data_Scraping\chromedriver.exe


images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'N

C:\Users\fayyo\AppData\Local\Temp\ipykernel_16200\1734802755.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path='C:/User/fayyo/chromedriver')
Incompatible release of chromedriver (version 111.0.5563.64) detected in PATH: c:\Users\GitHub\Bots_using_selenium\linkedin_data_Scraping\chromedriver.exe


images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
creating df


C:\Users\fayyo\AppData\Local\Temp\ipykernel_16200\1734802755.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path='C:/User/fayyo/chromedriver')
Incompatible release of chromedriver (version 111.0.5563.64) detected in PATH: c:\Users\GitHub\Bots_using_selenium\linkedin_data_Scraping\chromedriver.exe


images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
creating df


C:\Users\fayyo\AppData\Local\Temp\ipykernel_16200\1734802755.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path='C:/User/fayyo/chromedriver')
Incompatible release of chromedriver (version 111.0.5563.64) detected in PATH: c:\Users\GitHub\Bots_using_selenium\linkedin_data_Scraping\chromedriver.exe


images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'Non

C:\Users\fayyo\AppData\Local\Temp\ipykernel_16200\1734802755.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path='C:/User/fayyo/chromedriver')
Incompatible release of chromedriver (version 111.0.5563.64) detected in PATH: c:\Users\GitHub\Bots_using_selenium\linkedin_data_Scraping\chromedriver.exe


images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
images_error--> 'NoneType' object is not subscriptable
images_error--> 'NoneType' object is not subscriptable
salaries_error--> 'NoneType' object has no attribute 'text'
salaries_error-

In [6]:
df_teache=pd.DataFrame({
        "job_titles":job_titles,
        "job_description":list(job_desc_Math_teacher.values()),
        "images":images,
        "locations":locations,
        "salaries":salaries,
        "companies":companies
        }
    )
df_teache.to_csv(f"data_science{list(sectors_physics_and_astronomy.keys())[j]}.csv", sep=',', index=False)

In [10]:
len(list(job_desc_Math_teacher.values()))

25

In [8]:
# len(list(job_desc_Math_teacher.values()))
len(images)

29

In [35]:
len(list(job_desc_Math_teacher.values()))

29

In [1]:
import pandas as pd
csv_data = pd.read_csv('accountingBanen in accounting.csv')

json_data = csv_data.to_json(orient = 'records')

with open('json_data.json', 'w') as jsonfile:
    jsonfile.write(json_data)

In [36]:
df = pd.read_csv("Neuroscience_Banen in software & informatietechnologie_last.csv")

In [9]:
len(list(job_desc_Math_teacher.values()))

29

In [ ]:

    # pros=[]
    # cons=[]

    # driver.close()
    # driver = webdriver.Chrome(options=options)
    # site1 = "https://www.glassdoor.com/Search/results.htm?keyword=&locId=178&locT=N&locName=Netherlands"

    # for comp in companies:
    #     driver.get(site1)
    #     time.sleep(20)
    #     comp_input=driver.find_element(By.XPATH, '//*[@id="sc.keyword"] ')  
    #     time.sleep(5)
    #     comp_input.click()
    #     comp_input.clear()
    #     comp_input.send_keys(comp)
    #     time.sleep(5)
    #     try:
    #         search_button=driver.find_element(By.XPATH, '//*[@id="scBar"]/div/button')
    #         search_button.click()
    #         time.sleep(5)

    #         first_comp=driver.find_element(By.XPATH, '//*[@id="Discover"]/div/div/div[1]/div[1]/div[1]/a[1]')
    #         first_comp.click()
    #         time.sleep(5)
    #                                                             #eiCell cell reviews 
    #         comp_review_button=driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/div/div/div[1]/article[1]/div[1]/div[2]/div/div[2]/div/div[1]/div/a[1]')
    #         comp_review_button.click()
    #         time.sleep(5)
            
    #         initialScroll = 0
    #         finalScroll = 1000
    #         start = time.time()
    #         while True:
    #             driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
    #             initialScroll = finalScroll
    #             finalScroll += 1000
    #             time.sleep(3)
    #             end = time.time()
    #             if round(end - start) > 20:
    #                 break
    #         src = driver.page_source
    #         soup = BeautifulSoup(src, 'lxml')

    #         a =soup.find_all('p', {'class':"mt-0 mb-0 pb v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge v2__EIReviewDetailsV2__isCollapsed"})
            
    #         list_cons=[]
    #         list_pros=[]
    #         for i in a:
    #             try:
    #                 list_cons.append(i.find('span', {'data-test':"cons"}).text.strip())
    #             except Exception as e:
    #                 print(e)
    #         for i in a:
    #             try:
    #                 list_pros.append(i.find('span', {'data-test':"pros"}).text.strip())
    #             except Exception as e:
    #                 print(e)
    #         if len(list_cons)==0:
    #             list_cons.append('NG')
    #         if len(list_pros)==0:
    #             list_pros.append('NG')

    #         pros.append(list_pros)
    #         cons.append(list_cons)
    #     except Exception as e:
    #         print(e)
    #         pros.append("NG")
    #         cons.append("NG")

    # driver.close()


In [ ]:

df19=pd.read_csv('customer-service2.csv')
df20=pd.read_csv('customer-service3.csv')
df21=pd.read_csv('customer-service4.csv')
df22=pd.read_csv('customer-service5.csv')
df23=pd.read_csv('customer-service6.csv')
df24=pd.read_csv('customer-service7.csv')
df25=pd.read_csv('customer-service8.csv')
df26=pd.read_csv('customer-service9.csv')
df27=pd.read_csv('customer-service10.csv')
df28=pd.read_csv('customer-service11.csv')
df29=pd.read_csv('customer-service12.csv')
df30=pd.read_csv('customer-service13.csv')
df31=pd.read_csv('customer-service14.csv')




In [4]:
df32=pd.read_csv('data_analystBanen in software & informatietechnologie.csv')
df33=pd.read_csv('data_analystBanen in software & informatietechnologie1.csv')

df34=pd.read_csv('data_scienceBanen in software & informatietechnologie.csv')
df35=pd.read_csv('data_scienceBanen in software & informatietechnologie1.csv')
df36=pd.read_csv('data_scienceBanen in software & informatietechnologie2.csv')
df37=pd.read_csv('data_scienceBanen in software & informatietechnologie3.csv')
df38=pd.read_csv('data_scienceBanen in software & informatietechnologie4.csv')
df39=pd.read_csv('data_scienceBanen in software & informatietechnologie5.csv')
df40=pd.read_csv('data_scienceBanen in software & informatietechnologie6.csv')
df41=pd.read_csv('data_scienceBanen in software & informatietechnologie7.csv')
df42=pd.read_csv('data_scienceBanen in software & informatietechnologie8.csv')
df43=pd.read_csv('data_scienceBanen in software & informatietechnologie9.csv')

df44=pd.read_csv('developerBanen in software & informatietechnologie.csv')
df45=pd.read_csv('developerBanen in software & informatietechnologie1.csv')
df46=pd.read_csv('developerBanen in software & informatietechnologie2.csv')
df47=pd.read_csv('developerBanen in software & informatietechnologie3.csv')

df48=pd.read_csv('Neuroscience_Banen in software & informatietechnologie_last.csv')
df49=pd.read_csv('Neuroscience_Banen in software & informatietechnologie_last1.csv')
df50=pd.read_csv('Neuroscience_Banen in software & informatietechnologie_last2.csv')
df51=pd.read_csv('Neuroscience_Banen in software & informatietechnologie_last3.csv')

In [103]:
software = pd.concat([df32, df33, df34,df35,df36,df37,df38,df39,df40,df41,df42,df43,
                      df44,df45,df46,df47,df48,df49,df50,df51,]).reset_index(drop=True)
software['id']=range(0, len(software))

In [104]:
software_last=software.drop('job_description', axis=1).fillna("Not Given")

In [30]:
df1=pd.read_csv("accountingBanen in accounting.csv")
df2=pd.read_csv("accountingBanen in accounting2.csv")
df3=pd.read_csv("accountingBanen in accounting3.csv")
df4=pd.read_csv("accountingBanen in accounting4.csv")
df5=pd.read_csv("accountingBanen in accounting5.csv")

df52=pd.read_csv('financeBanen in finance & accounting.csv')
df53=pd.read_csv('financeBanen in finance & accounting3.csv')
df54=pd.read_csv('financeBanen in finance & accounting4.csv')
df55=pd.read_csv('financeBanen in finance & accounting5.csv')
df56=pd.read_csv('financeBanen in finance & accounting6.csv')
df57=pd.read_csv('financeBanen in finance & accounting7.csv')
df58=pd.read_csv('financeBanen in finance & accounting8.csv')
df59=pd.read_csv('financeBanen in finance & accounting9.csv')
df60=pd.read_csv('financeBanen in finance & accounting10.csv')
df61=pd.read_csv('financeBanen in finance & accounting11.csv')
df62=pd.read_csv('financeBanen in finance & accounting12.csv')
df63=pd.read_csv('financeBanen in finance & accounting13.csv')
df64=pd.read_csv('financeBanen in finance & accounting14.csv')
df65=pd.read_csv('financeBanen in finance & accounting15.csv')
df66=pd.read_csv('financeBanen in finance & accounting16.csv')

df67=pd.read_csv('financeBanen in finance17.csv')
df68=pd.read_csv('financeBanen in finance18.csv')
df69=pd.read_csv('financeBanen in finance19.csv')
df70=pd.read_csv('financeBanen in finance20.csv')
df71=pd.read_csv('financeBanen in finance21.csv')
df72=pd.read_csv('financeBanen in finance22.csv')

In [105]:
finance = pd.concat([df1, df2, df3,df4,df5,df52,df53,df54,df55,df56,df57,df58,
                      df59,df60,df61,df62,df63,df64,df65,df66,df67,df68,df69,df70,
                      df71,df72]).reset_index(drop=True)
finance['id']=range(0, len(finance))

In [106]:
finance_last=finance.drop('job_description', axis=1).fillna("Not Given")

In [33]:

df73=pd.read_csv('product-worker2.csv')
df74=pd.read_csv('product-worker3.csv')
df75=pd.read_csv('product-worker4.csv')
df76=pd.read_csv('product-worker5.csv')
df77=pd.read_csv('product-worker6.csv')
df78=pd.read_csv('product-worker7.csv')
df79=pd.read_csv('product-worker8.csv')
df80=pd.read_csv('product-worker9.csv')
df81=pd.read_csv('product-worker10.csv')
df82=pd.read_csv('product-worker11.csv')

df83=pd.read_csv('supply-chain-logistics2.csv')
df84=pd.read_csv('supply-chain-logistics3.csv')
df85=pd.read_csv('supply-chain-logistics4.csv')
df86=pd.read_csv('supply-chain-logistics5.csv')
df87=pd.read_csv('supply-chain-logistics6.csv')
df6=pd.read_csv('assembly-product2.csv')
df7=pd.read_csv('assembly-product3.csv')
df8=pd.read_csv('assembly-product4.csv')

df9=pd.read_csv('construction3.csv')
df10=pd.read_csv('construction4.csv')
df11=pd.read_csv('construction5.csv')
df12=pd.read_csv('construction6.csv')
df13=pd.read_csv('construction7.csv')
df14=pd.read_csv('construction8.csv')
df15=pd.read_csv('construction9.csv')
df16=pd.read_csv('construction10.csv')

df17=pd.read_csv("constructionBanen in construction1.csv")
df18=pd.read_csv("constructionBanen in construction2.csv")

In [140]:
sector3 = pd.concat([df1, df2, df3,df4,df5,df52,df53,df54,df55,df56,df57,df58,
                      df59,df60,df61,df62,df63,df64,df65,df66,df67,df68,df69,df70,
                      df71,df72]).reset_index(drop=True)
sector3['id']=range(0, len(sector3))
sector3_last=sector3.drop('job_description', axis=1).fillna("Not Given")
# sector3_last=sector3_last.drop('job_description', axis=1)

In [115]:
d0=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur0.csv')
d1=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur1.csv')
d2=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur2.csv')
d3=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur3.csv')
d4=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur4.csv')
d5=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur5.csv')
d6=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur6.csv')
d7=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur7.csv')
d8=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur8.csv')
d9=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur9.csv')
d10=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur10.csv')
d11=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur11.csv')
a12=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur12.csv')
a13=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur13.csv')
a14=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur14.csv')
a15=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur15.csv')
a16=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur16.csv')
a17=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur17.csv')
a18=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur18.csv')
a19=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur19.csv')
a20=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur20.csv')
a21=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur21.csv')
a22=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur22.csv')
a23=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur23.csv')
a24=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur24.csv')
a25=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur25.csv')
a26=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur26.csv')
a27=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur27.csv')
a28=pd.read_csv('Banen_in_media_design_cultuurmedia, design & cultuur28.csv')

In [111]:
c2=pd.read_csv('healthcare2.csv')
c3=pd.read_csv('healthcare3.csv')
c4=pd.read_csv('healthcare4.csv')
c5=pd.read_csv('healthcare5.csv')
c6=pd.read_csv('healthcare6.csv')
c7=pd.read_csv('healthcare7.csv')
c8=pd.read_csv('healthcare8.csv')
c9=pd.read_csv('healthcare9.csv')
c10=pd.read_csv('healthcare10.csv')
c11=pd.read_csv('healthcare11.csv')
c12=pd.read_csv('healthcare12.csv')
c13=pd.read_csv('healthcare13.csv')
c14=pd.read_csv('healthcare14.csv')
c15=pd.read_csv('healthcare15.csv')
c16=pd.read_csv('healthcare16.csv')
c17=pd.read_csv('healthcare17.csv')
c18=pd.read_csv('healthcare18.csv')
c19=pd.read_csv('healthcare19.csv')
c20=pd.read_csv('healthcare20.csv')
c21=pd.read_csv('healthcare21.csv')
c22=pd.read_csv('healthcare22.csv')
c23=pd.read_csv('healthcare23.csv')
c24=pd.read_csv('healthcare24.csv')

In [141]:
sector4 = pd.concat([
    c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19,c20,c21,c22,c23,c24
]).reset_index(drop=True)
sector4['id']=range(0, len(sector4))
sector4_last=sector4.drop('job_description', axis=1).fillna("Not Given")

In [142]:
sector5 = pd.concat([d0, d1, d2, d3,d4,d5,d6,d7,d8,d9,d10,d11,a12,a13,a14,
                    a15,a16,a17,a18,a19,a20,a21,a22,a23,a24,a25,a26,a27,a28
                    ]).reset_index(drop=True)
sector5['id']=range(0, len(sector5))
sector5_last=sector5.drop('job_description', axis=1).fillna("Not Given")

In [118]:
b1=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service1.csv')
b2=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service2.csv')
b3=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service3.csv')
b4=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service4.csv')
b5=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service5.csv')
b6=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service6.csv')
b7=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service7.csv')
b8=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service8.csv')
b9=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service9.csv')
b10=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service10.csv')
b11=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service11.csv')
b12=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service12.csv')
b13=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service13.csv')
b14=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service14.csv')
b15=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service15.csv')
b16=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service16.csv')
b17=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service17.csv')
b18=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service18.csv')
b19=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service19.csv')
b20=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service20.csv')
b21=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service21.csv')
b22=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service22.csv')
b23=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service23.csv')
b24=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service24.csv')
b25=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service25.csv')
b26=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service26.csv')
b27=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service27.csv')
b28=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service28.csv')
b29=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service29.csv')
b30=pd.read_csv('Banen_in_supply_chain__logistiek_and_Banen_in_klantenondersteuning-serviceBanen in supply chain & logistiek and Banen in klantenondersteuning-service30.csv')

In [143]:
sector6 = pd.concat([ b1, b2, b3,b4,b5,b6,b7,b8,b9,b10,b11,b12,b13,b14,
                    b15,b16,b17,b18,b19,b20,b21,b22,b23,b24,b25,b26,b27,b28,b29,b30
                    ]).reset_index(drop=True)
sector6['id']=range(0, len(sector6))
sector6_last=sector6.drop('job_description', axis=1).fillna("Not Given")

In [156]:
software_last['job_description']=software['job_description']
finance_last['job_description']=finance['job_description']
sector3_last['job_description']=sector3['job_description']
sector4_last['job_description']=sector4['job_description']
sector5_last['job_description']=sector5['job_description']
sector6_last['job_description']=sector6['job_description']

In [157]:
data={
    "Banen in software & informatietechnologie":software_last.to_dict(orient="records"),
    "Banen in finance & accounting":finance_last.to_dict(orient='records'),
    "Banen in bouw & architectuur and Banen in productie, montage":sector3_last.to_dict(orient='records'),
    "Banen in zorg & welzijn adn Banen in gezondheidszor":sector4_last.to_dict(orient='records'),
    "Banen in media, design & cultuur":sector5_last.to_dict(orient="records"),
    "Banen in supply chain & logistiek and Banen in klantenondersteuning & -service":sector6_last.to_dict(orient="records"),
}

In [158]:
for key, value in data.items():
    if isinstance(value, list):
        for item in value:
            for k, v in item.items():
                try:
                    if '★' in item[k]:
                        item[k]=item[k][0:-5]
                    if '€' in item[k]:
                        item[k]=item[k].replace('€', '')
                    if ' ' in item[k]:
                        item[k]=item[k].replace(' ', '')
                    if '–' in item[k]:
                        item[k]=item[k].replace('–', '-')
                except Exception as e:
                    print(e)
                    continue
                # item[k] = v.encode().decode('unicode_escape')

argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is not iterable
argument of type 'int' is

In [159]:
import json
json_data2 =json.dumps(data, indent=4, ensure_ascii=False)
with open("data.json", "w", encoding='utf-8') as file:
    file.write(json_data2)

In [161]:
json_file = 'data.json'
with open(json_file, encoding="utf8") as file:
    data = json.load(file)
    software_data=data["Banen in software & informatietechnologie"]
    with open("software_data.json", "w", encoding='utf-8') as file:
        file.write(software_data)

TypeError: write() argument must be str, not list

In [164]:
with open('data.json', encoding='utf-8') as file:
    data = json.load(file)

# Iterate over each key in the JSON data
for key in data:
    # Create a new JSON file for each key
    filename = key + '.json'
    with open(filename, 'w') as file:
        # Write the corresponding value to the new JSON file
        json.dump(data[key], file)

: 

In [ ]:
"Banen in software & informatietechnologie"

In [133]:
software_desc_id=pd.DataFrame()
finance_desc_id=pd.DataFrame()
sector3_desc_id=pd.DataFrame()
sector4_desc_id=pd.DataFrame()
sector5_desc_id=pd.DataFrame()
sector6_desc_id=pd.DataFrame()

In [152]:
software_desc_id['job_description']=software['job_description']
finance_desc_id['job_description']=finance['job_description']
sector3_desc_id['job_description']=sector3['job_description']
sector4_desc_id['job_description']=sector4['job_description']
sector5_desc_id['job_description']=sector5['job_description']
sector6_desc_id['job_description']=sector6['job_description']

In [153]:
software_desc_id['id']=software['id']
finance_desc_id['id']=finance['id']
sector3_desc_id['id']=sector3['id']
sector4_desc_id['id']=sector4['id']
sector5_desc_id['id']=sector5['id']
sector6_desc_id['id']=sector6['id']

In [154]:
data_id={
    "Banen in software & informatietechnologie":software_desc_id.to_dict(orient="records"),
    "Banen in finance & accounting":finance_desc_id.to_dict(orient='records'),
    "Banen in bouw & architectuur and Banen in productie, montage":sector3_desc_id.to_dict(orient='records'),
    "Banen in zorg & welzijn adn Banen in gezondheidszor":sector4_desc_id.to_dict(orient='records'),
    "Banen in media, design & cultuur":sector5_desc_id.to_dict(orient="records"),
    "Banen in supply chain & logistiek and Banen in klantenondersteuning & -service":sector6_desc_id.to_dict(orient="records"),
}

In [155]:
import json
json_data_id =json.dumps(data_id, indent=4, ensure_ascii=False)
with open("data_id_desc.json", "w", encoding='utf-8') as file:
    file.write(json_data_id)